In [ ]:
! pip install transformers
! pip install tqdm
! pip install -U sentence-transformers
! pip install wandb


import h5py
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import pandas as pd
import datetime
from pathlib import Path
from transformers import AdamW, get_linear_schedule_with_warmup

from html import escape
from transformers import BertTokenizer, TFBertModel
from keras.layers import Masking
from keras.layers import Input, LSTM, Embedding, Dense, TimeDistributed, Bidirectional
from keras.models import Model
import tensorflow as tf
import datetime
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Lambda
import tensorflow as tf
from transformers import TFBertModel
from tensorflow.keras.layers import TimeDistributed, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras import Input
import wandb
from tensorflow.keras.layers import LSTM
import wandb
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
from sentence_transformers import SentenceTransformer
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import classification_report
from google.colab import drive
import pickle
from PIL import Image
import io
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score, recall_score
import time
from pathlib import Path
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F
from nltk.tokenize import sent_tokenize
from torch.nn.functional import softmax
import nltk
nltk.download('punkt')
from sentence_transformers import SentenceTransformer
import torch.nn as nn
import torch
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.tokenize import sent_tokenize
from torch.nn.functional import softmax
import random
from nltk.tokenize import sent_tokenize
from torch.nn.functional import softmax
from tensorflow.keras.layers import Lambda
import tensorflow as tf
import pickle

from tensorflow.keras.layers import Reshape

from google.colab import drive


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=993ac0b746802a19c5d47ba6b6616ec306ddad229ac7cf0a08fdbcfdcaf98c77
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 15.6 MB/s eta 0

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
drive.mount('/content/drive')
!wandb login

Mounted at /content/drive
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
## FULL DATA

X_path = "/content/drive/MyDrive/thesis_files/X_SBERT_FULLDATA_Final.pkl"
y_path = "/content/drive/MyDrive/thesis_files/y_SBERT_FULLDATA_Final.pkl"



In [ ]:
## MEDIUM



X_path = "/content/drive/MyDrive/thesis_files/X_SBERT_medium_data_third_150.pkl"
y_path = "/content/drive/MyDrive/thesis_files/y_SBERT_medium_data_third_150.pkl"



In [ ]:
# Load 'X' data from pickle file
with open(X_path, 'rb') as pkl_file:
    X = pickle.load(pkl_file)

# Load 'y' data from pickle file
with open(y_path, 'rb') as pkl_file:
    y = pickle.load(pkl_file)

In [ ]:
## CLASSES

class HierarchicalSentenceBertForSentenceTagging(nn.Module):
    def __init__(self, sbert_model_name, num_classes, dropout_rate=0.1, lstm_hidden_size=100, device=torch.device('cpu')):
        super(HierarchicalSentenceBertForSentenceTagging, self).__init__()

        self.sbert_model = SentenceTransformer(sbert_model_name)
        self.lstm = nn.LSTM(self.sbert_model.get_sentence_embedding_dimension(), lstm_hidden_size, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(2*lstm_hidden_size, num_classes)
        self.device = device

    def forward(self, document_sentences):
        # Get sentence embeddings for all sentences in all documents
        sentence_embeddings = [self.sbert_model.encode(doc_sentences, convert_to_tensor=True) for doc_sentences in document_sentences]
        sentence_embeddings = [torch.unsqueeze(se, 0).to(self.device) for se in sentence_embeddings]  # Unsqueeze and move each tensor to the device
        sentence_embeddings = torch.cat(sentence_embeddings, 0)  # Concatenate along the new dimension

        # Apply LSTM, Dropout, and Linear layer
        lstm_outputs, _ = self.lstm(sentence_embeddings)
        lstm_outputs = self.dropout(lstm_outputs)
        logits = self.classifier(lstm_outputs)

        return logits

# Custom dataset
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        return (self.data[index], self.labels[index])

    def __len__(self):
        return len(self.data)

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2, ignore_index=None):
        super().__init__()
        self.alpha = alpha
        if alpha is not None:
            if isinstance(alpha, (list, tuple)):
                self.alpha = torch.tensor(alpha).to(device)  # Move alpha to the device
            elif isinstance(alpha, torch.Tensor):
                self.alpha = alpha.to(device)  # Move alpha to the device
        self.gamma = gamma
        self.ignore_index = ignore_index

    def forward(self, input, target):
        # Compute the negative log-likelihood
        logpt = F.cross_entropy(input, target, reduction='none', ignore_index=self.ignore_index)
        pt = torch.exp(-logpt)

        # Compute the focal loss
        if self.alpha is not None:
            at = self.alpha.gather(0, target.data.view(-1))
            logpt = logpt * at

        focal_loss = (1-pt)**self.gamma * logpt

        return focal_loss.mean()

In [ ]:
## FUNCTIONS

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

def calculate_class_weights(y_train, device, class_h_factor=1.0):
    # Concatenate all the labels
    y_all = np.concatenate([doc_labels.cpu().numpy() for doc_labels in y_train])
    #y_all = np.concatenate(y_train)

    # Count unique labels
    unique_labels, counts = np.unique(y_all, return_counts=True)

    # Compute class weights and apply the class_h_factor to the second class
    weights = 1.0 / counts
    weights_dict = dict(zip(unique_labels, weights))

    # Modify the weight of the highlight / 2 (third by zero index) class
    if 1 in weights_dict:
        weights_dict[2] *= class_h_factor

    # Build weights list
    weights_list = [weights_dict[i] if i in weights_dict else 0 for i in range(min(unique_labels), max(unique_labels)+1)]

    # Convert to tensor
    class_weights = torch.tensor(weights_list).float().to(device)

    return class_weights, weights_dict

def split_data(X, y, train_size, val_size, test_size, random_state):
    assert train_size + val_size + test_size == 1, "train_size, val_size, and test_size must sum up to 1"

    # Split data into temporary and test datasets
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Adjust train_size to account for the previous split
    train_size_adjusted = train_size / (train_size + val_size)

    # Split the temporary dataset into train and validation datasets
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=1 - train_size_adjusted, random_state=random_state
    )

    return X_train, X_val, X_test, y_train, y_val, y_test


def set_up_data(X, y, batch_size, collate_fn, train_size, val_size, test_size, random_state):
    # Assuming split_data is a function that splits your data and returns X, y train, val, and test
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y, train_size, val_size, test_size, random_state)
    # Convert labels to tensors, convert to long integer type, and move to device
    y_train = torch.tensor(y_train).long().to(device)
    y_val = torch.tensor(y_val).long().to(device)
    y_test = torch.tensor(y_test).long().to(device)

    # Create instances of MyDataset for each of the splits
    train_dataset = MyDataset(X_train, y_train)
    val_dataset = MyDataset(X_val, y_val)
    test_dataset = MyDataset(X_test, y_test)

    # Create DataLoader instances for each of the datasets
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=False)

    print(f'X_train len: {len(X_train)}')
    print(f'X_val len: {len(X_val)}')
    print(f'X_test len: {len(X_test)}')
    # Return the data loaders
    return train_dataloader, val_dataloader, test_dataloader, y_train


def collate_fn(batch):
    sentences, labels = zip(*batch)  # unzip the batch
    sentences = list(sentences)

    # Convert lists of labels to tensors. Before that, convert each list of labels to a tensor.
    labels = [torch.tensor(l) for l in labels]
    labels = pad_sequence(labels, batch_first=True, padding_value=0)  # Assuming 0 will be ignored by the loss function.
    return sentences, labels

def setup_model(sbert_model_name, num_classes, device=None):
    # Model creation
    model = HierarchicalSentenceBertForSentenceTagging(sbert_model_name, num_classes, device=device)

    # If a device is not provided, use CUDA if available
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)

    # Log model information to Weights and Biases
    wandb.log({'model_name': sbert_model_name})

    return model


def set_up_loss_optimizer(model, train_dataloader, num_epochs, learning_rate, eps, loss_type, use_scheduler=True, num_warmup_steps=0, class_weights=None, alpha=None, gamma=None):


    # Only optimize parameters that require gradients (i.e., unfrozen parameters)
    optimizer = AdamW([param for param in model.parameters() if param.requires_grad], lr=learning_rate, eps=eps)

    if loss_type.lower() == 'focal':
        # Use the provided alpha and gamma parameters for the Focal Loss
        loss_function = FocalLoss(alpha=alpha, gamma=gamma, ignore_index=0)
    elif loss_type.lower() == 'cross_entropy':
        loss_function = nn.CrossEntropyLoss(weight=class_weights, ignore_index=0)
    else:
        raise ValueError("Invalid loss_type. Expected 'focal' or 'cross_entropy'")

    if use_scheduler:
        total_steps = len(train_dataloader) * num_epochs
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps)
    else:
        scheduler = None

    return optimizer, loss_function, scheduler


def train_model(model, train_dataloader, optimizer, loss_function, epoch, scheduler=None):
    epoch += 1  # to not zero index
    model.train()  # Set the model to training mode
    total_loss = 0
    total_accuracy = 0
    total_recall = 0
    total_steps = 0

    progress_bar = tqdm(train_dataloader, desc=f'Train Epoch {epoch}')

    for batch in progress_bar:
        optimizer.zero_grad()
        sentences, labels = batch

        outputs = model(sentences)  # Sentences remain as a list

        batch_loss = 0
        batch_predicted = []
        batch_labels = []

        for doc_output, doc_label in zip(outputs, labels):
            loss = loss_function(doc_output.view(-1, doc_output.shape[-1]), doc_label.view(-1))
            batch_loss += loss  # Here, no .item()

            _, predicted = torch.max(doc_output, 1)
            batch_predicted.extend(predicted.view(-1).cpu().numpy())
            batch_labels.extend(doc_label.view(-1).cpu().numpy())

        batch_loss.backward()  # Here, the backward() call
        optimizer.step()  # Here, the step() call

        total_loss += batch_loss.item()  # Update total_loss after the optimizer step
        total_steps += 1

        # logging the loss?
        wandb.log({
                'epoch': epoch,
                'Train/Loss': batch_loss / total_steps,
                'Train/Accuracy': total_accuracy / total_steps,
                'Train/Recall': total_recall / total_steps
            })

        # Update progress bar
        progress_bar.set_postfix({
            'loss': batch_loss / total_steps,
             'accuracy': total_accuracy / total_steps,
            'recall': total_recall / total_steps
            })
        # Calculate accuracy and recall
        total_accuracy += accuracy_score(batch_labels, batch_predicted)
        total_recall += recall_score(batch_labels, batch_predicted, average='micro')



    # Compute the average loss over the training data.
    avg_train_loss = total_loss / total_steps
    avg_train_accuracy = total_accuracy / total_steps
    avg_train_recall = total_recall / total_steps

    return avg_train_loss, avg_train_accuracy, avg_train_recall


def validate_model(model, val_dataloader, loss_function, epoch):
    epoch += 1
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_accuracy = 0
    total_recall = 0
    total_steps = 0

    progress_bar = tqdm(val_dataloader, desc=f'Val Epoch {epoch}')
    with torch.no_grad():
        for batch in progress_bar:
            sentences, labels = batch
            outputs = model(sentences)  # Sentences remain as a list

            batch_loss = 0
            batch_predicted = []
            batch_labels = []

            for doc_output, doc_label in zip(outputs, labels):
                loss = loss_function(doc_output.view(-1, doc_output.shape[-1]), doc_label.view(-1))
                batch_loss += loss #.item()

                _, predicted = torch.max(doc_output, 1)
                batch_predicted.extend(predicted.view(-1).cpu().numpy())
                batch_labels.extend(doc_label.view(-1).cpu().numpy())

            # Calculate accuracy and recall
            total_accuracy += accuracy_score(batch_labels, batch_predicted)
            total_recall += recall_score(batch_labels, batch_predicted, average='micro')

            total_steps += 1
            total_loss += batch_loss.item()  # Update total_loss after all calculations


            # Log metrics to wandb
            wandb.log({
                'epoch': epoch,
                'Validation/Loss': batch_loss / total_steps,
                'Validation/Accuracy': total_accuracy / total_steps,
                'Validation/Recall': total_recall / total_steps
            })

            # Update progress bar
            progress_bar.set_postfix({
                'loss': batch_loss / total_steps,
                'accuracy': total_accuracy / total_steps,
                'recall': total_recall / total_steps
            })

    # Compute the average loss over the validation data.
    avg_val_loss = total_loss / total_steps
    avg_val_accuracy = total_accuracy / total_steps
    avg_val_recall = total_recall / total_steps

    return avg_val_loss, avg_val_accuracy, avg_val_recall



def test_model(model, test_dataloader):

    model.eval()  # Set the model to evaluation mode
    total_steps = 0

    all_predicted = []
    all_labels = []

    progress_bar = tqdm(test_dataloader, desc=f'Test')
    with torch.no_grad():
        for batch in progress_bar:
            sentences, labels = batch
            outputs = model(sentences)  # Sentences remain as a list

            for doc_output, doc_label in zip(outputs, labels):
                doc_output_reshaped = doc_output.view(-1, doc_output.shape[-1])
                doc_label_reshaped = doc_label.view(-1)
                _, predicted = torch.max(doc_output, 1)
                all_predicted.extend(predicted.view(-1).cpu().numpy())
                all_labels.extend(doc_label.view(-1).cpu().numpy())

            total_steps += 1

        # Calculate accuracy and recall after all predictions are gathered
        total_accuracy = accuracy_score(all_labels, all_predicted)
        total_recall = recall_score(all_labels, all_predicted, average='micro')

        # Update progress bar
        progress_bar.set_postfix({
            'accuracy': total_accuracy,
            'recall': total_recall
        })

    # Calculate and print the classification report
    classification_report_str = classification_report(all_labels, all_predicted, zero_division=1)
    classification_report_dict = classification_report(all_labels, all_predicted, zero_division=1, output_dict=True)

    # Log metrics and the classification report to wandb
    wandb.log({
        'Test/Accuracy': total_accuracy,
        'Test/Recall': total_recall,
        'Test/Classification Report': classification_report_dict
    })

    # Convert the classification report to a text plot
    fig, ax = plt.subplots()
    ax.text(0.5, 0.5, classification_report_str, horizontalalignment='center', verticalalignment='center')
    plt.axis('off')

    # Save the plot to a BytesIO object
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)

    # Load the image from the BytesIO object and log it to wandb
    img = Image.open(buf)
    wandb.log({'Test/Classification Report Image': [wandb.Image(img, caption='Classification Report')]})

    # Close the BytesIO object
    buf.close()

    return total_accuracy, total_recall, all_labels, all_predicted


def save_model_to_drive(model, model_name):
    # Set the save path
    save_path = "/content/drive/MyDrive/thesis_files/SBERT/" + model_name

    # Save the model
    torch.save(model.state_dict(), save_path)

    print(f"Model saved at {save_path}")



def inference_testing(json_file_path, model, name):
    html_files = []
    # Load data from json
    data_df = pd.read_json(json_file_path)

    # Get current date
    date_str = datetime.date.today().strftime('%Y%m%d')

    # Create a new directory for this run's HTML files
    output_dir = f'/content/drive/MyDrive/thesis_files/SBERT/{name}_{date_str}'
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Iterate over documents in the DataFrame
    for idx, row in data_df.iterrows():
        title = row['title']
        content = row['content']

        # Run inference with the model
        sentence_label_pairs = run_inference(model, content)

        # Generate HTML document
        html_content = html_highlight(sentence_label_pairs)

        # Generate HTML file path
        html_file_path = os.path.join(output_dir, f'{title}.html')

        # Save HTML to a file
        with open(html_file_path, 'w') as f:
            f.write(html_content)

        # Add HTML file path to the list
        html_files.append(html_file_path)

    return html_files


def html_highlight(sentence_label_pairs, color="yellow", sentences_per_paragraph=5):
    """
    Generate HTML for the given sentences and labels.

    Args:
        sentence_label_pairs (List[Tuple[str, str]]): The list of pairs of sentences and labels.
        color (str, optional): The highlight color for 'highlight' labels.
            Defaults to "yellow".
        sentences_per_paragraph (int, optional): The number of sentences per paragraph.
            Defaults to 5.

    Returns:
        str: The generated HTML string.
    """
    html = []
    sentence_count = 0
    for sentence, label in sentence_label_pairs:
        if label == 'highlight':
            html.append(f'<mark style="background-color: {color};">{escape(sentence)}</mark>')
        else:
            html.append(escape(sentence))

        sentence_count += 1
        if sentence_count == sentences_per_paragraph:
            html.append('<br/><br/>')
            sentence_count = 0

    html_text = ' '.join(html)

    # Add HTML and CSS to resemble Medium blog post style
    html_text = f"""
    <!DOCTYPE html>
    <html>
    <head>
    <style>
    body {{
        font-family: 'Times New Roman', Times, serif;
        color: #444;
        background-color: #FAFAFA;
        margin: 0;
        padding: 0;
    }}
    .content {{
        max-width: 800px;
        margin: 0 auto;
        padding: 2em;
        font-size: 18px;
        line-height: 1.6;
    }}
    mark {{
        background-color: #FFF9C4;
        padding: 0.2em;
    }}
    </style>
    </head>
    <body>
    <div class="content">
    {html_text}
    </div>
    </body>
    </html>
    """

    return html_text





def run_inference(model, document):

    label_tokenizer = Tokenizer(filters='', lower=False)

    # Explicitly set the word_index
    label_tokenizer.word_index = {'normaltext': 1, 'highlight': 2}
    # This assumes 'X' is used for subword tokens and 'PAD' is used for padding tokens
    label_tokenizer.word_index['X'] = len(label_tokenizer.word_index) + 1
    label_tokenizer.word_index['PAD'] = 0

    # Tokenize document into sentences
    sentences = sent_tokenize(document)

    # Process sentences to match the model's expected input format
    document_sentences = [sentences]  # Model expects a list of lists of sentences

    # Run model
    model.eval()
    with torch.no_grad():
        output = model(document_sentences)
        predictions = torch.argmax(output, dim=2)
        probabilities = softmax(output, dim=2)

    # Convert tensor of predictions to list
    predictions = predictions.cpu().numpy().tolist()

    # Convert label indices back to label strings
    index_to_tag = {v: k for k, v in label_tokenizer.word_index.items()}
    label_predictions = [[index_to_tag[pred] for pred in doc_preds] for doc_preds in predictions]

    # Pair each sentence with its predicted label
    sentence_label_pairs = [(sent, label) for sent, label in zip(sentences, label_predictions[0])]
    print(sentence_label_pairs)
    return sentence_label_pairs



In [ ]:
## SETUP


# Set up your Weights & Biases project
run_name = "name"
wandb.init(project="project", name=run_name)

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wandb.log({'Device': str(device)})

SEED = 42

wandb: Currently logged in as: jf-frommann (bitdat-com). Use `wandb login --relogin` to force relogin


In [ ]:
## RUN


set_seed(SEED)
wandb.config.seed = SEED

# Data and Device Management
train_size = 0.8
val_size = 0.1
test_size = 0.1


# Convert labels to tensors, convert to long integer type, and move to device


# Set hyperparameters
sbert_model_name = 'all-MiniLM-L6-v2'
data_set_type='medium' # 'all' or 'medium'
loss_type='focal' # 'focal' or 'cross_entropy'

num_epochs = 3
batch_size = 32

learning_rate = 0.001
eps = 1e-8

num_classes = 3
class_h_factor = 1

min_alpha= 0.012
max_alpha = 1- min_alpha
print(max_alpha)
focal_alpha=[0,min_alpha,max_alpha,0]
focal_gamma=2.0

use_scheduler=False

# Log hyperparameters
wandb.config.update({
    "learning_rate" : learning_rate,
    "sbert_model_name" : sbert_model_name,
    "eps": eps,
    "num_epochs": num_epochs,
    "batch_size": batch_size,
    "sbert_model_name": sbert_model_name,
    "class_h_factor" : class_h_factor,
    "use_scheduler" : use_scheduler,
    "num_classes": num_classes,
    "data_set_type" : data_set_type,
    "loss_type": loss_type,
    "train_size": train_size,
    "val_size": val_size,
    "test_size": test_size,
    "min_alpha": min_alpha
})

# Set-up data loaders
train_dataloader, val_dataloader, test_dataloader, y_train = set_up_data(X, y, batch_size, collate_fn, train_size, val_size, test_size, random_state=SEED)

# Set-up the model
model = setup_model(sbert_model_name, num_classes, device)

# Calculate class weights for the loss function
class_weights, _ = calculate_class_weights(y_train, device, class_h_factor)

# Set-up loss function, optimizer, and scheduler
optimizer, loss_function, scheduler = set_up_loss_optimizer(model, train_dataloader, num_epochs, learning_rate, eps, loss_type, use_scheduler=use_scheduler, num_warmup_steps=0, class_weights=class_weights, alpha=focal_alpha, gamma=focal_gamma)

# Train and validate the model
for epoch in range(num_epochs):
    train_loss, train_accuracy, train_recall = train_model(model, train_dataloader, optimizer, loss_function, epoch, scheduler)
    val_loss, val_accuracy, val_recall = validate_model(model, val_dataloader, loss_function, epoch)

# Test the model
test_accuracy, test_recall, all_labels, all_predicted = test_model(model, test_dataloader)

# save the model
model_name = run_name + '.pt'
save_model_to_drive(model, model_name) # set-up

html_files = inference_testing('/content/drive/MyDrive/thesis_files/inference_docs_file.json', model, run_name)


# Upload each HTML file to Weights & Biases
for file in html_files:
    wandb.save(file)

wandb.finish()
